## Partie 1 : Installation et Configuration (15 min)

### 1.2 Importation des bibliothèques nécessaires


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# InfluxDB
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMap, MarkerCluster

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### 1.3 Configuration InfluxDB

In [ ]:
# Configuration InfluxDB
INFLUX_URL = "http://localhost:8086" 
INFLUX_TOKEN = "YIOy3XHBmCpYG0jIg3vnqC5c73RTQv-kJWHAh6JVtro1sEYIkTTNRIKjENP_dfLFd5xBT33WSswyIgkdftKylA=="  # Généré dans InfluxDB UI
INFLUX_ORG = "fil-A3-back-bigData"
INFLUX_BUCKET = "animal-tracking"  

# Connexion au client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)

# APIs
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

print("Client InfluxDB connecté")
print(f"Bucket: {INFLUX_BUCKET}")

✅ Client InfluxDB connecté
📊 Bucket: animal-tracking
